In [2]:
from pathlib import Path
from time import strftime

def get_run_logidir(root_logdir="my_logs"):
    return Path(root_logdir)/strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logidir()
run_logdir

WindowsPath('my_logs/run_2024_02_01_09_26_41')

In [3]:
import tensorflow as tf

class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs) # needed to support nameing the model
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)
        
    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output
    
model = WideAndDeepModel(30, activation="relu", name="my_cool_model")

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

housing = pd.read_csv('data/housing.csv')
housing.dropna(inplace=True)
housing.drop(columns=['ocean_proximity'], inplace=True)

X = housing.drop(columns=['median_house_value'])
y = housing['median_house_value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train_wide, X_train_deep = X_train.iloc[:, :5], X_train.iloc[:, 2:]
X_test_wide, X_test_deep = X_test.iloc[:, :5], X_test.iloc[:, 2:]
X_new_wide, X_new_deep = X_test_wide.iloc[:3], X_test_deep.iloc[:3]

In [5]:
model.compile(
    loss = ("mse","mse"),
    loss_weights= (0.9, 0.1),
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

tensorboard_callbacks = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))
history = model.fit((X_train_wide, X_train_deep), (y_train,y_train), epochs=10, validation_split=0.1, callbacks=tensorboard_callbacks)

Epoch 1/10
489/489 [==============================] - 4s 5ms/step - loss: 39400357888.0000 - output_1_loss: 39226839040.0000 - output_2_loss: 40962347008.0000 - output_1_root_mean_squared_error: 198057.6719 - output_2_root_mean_squared_error: 202391.5625 - val_loss: 27595368448.0000 - val_output_1_loss: 27568791552.0000 - val_output_2_loss: 27834611712.0000 - val_output_1_root_mean_squared_error: 166038.5312 - val_output_2_root_mean_squared_error: 166837.0781
Epoch 2/10
489/489 [==============================] - 2s 3ms/step - loss: 27272925184.0000 - output_1_loss: 27265245184.0000 - output_2_loss: 27342077952.0000 - output_1_root_mean_squared_error: 165121.9062 - output_2_root_mean_squared_error: 165354.4062 - val_loss: 27300452352.0000 - val_output_1_loss: 27294896128.0000 - val_output_2_loss: 27350454272.0000 - val_output_1_root_mean_squared_error: 165211.6719 - val_output_2_root_mean_squared_error: 165379.7344
Epoch 3/10
489/489 [==============================] - 2s 3ms/step - loss

In [8]:
# changing the learning rate
model.compile(
    loss = ("mse","mse"),
    loss_weights= (0.9, 0.1),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.003),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

tensorboard_callbacks = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200), update_freq='epoch')
history = model.fit((X_train_wide, X_train_deep), (y_train,y_train), epochs=10, validation_split=0.1, callbacks=tensorboard_callbacks)

Epoch 1/10
489/489 [==============================] - 3s 4ms/step - loss: 6715909120.0000 - output_1_loss: 6715616256.0000 - output_2_loss: 6718506496.0000 - output_1_root_mean_squared_error: 81948.8672 - output_2_root_mean_squared_error: 81966.5000 - val_loss: 7080589824.0000 - val_output_1_loss: 7079343616.0000 - val_output_2_loss: 7091808768.0000 - val_output_1_root_mean_squared_error: 84138.8359 - val_output_2_root_mean_squared_error: 84212.8750
Epoch 2/10
489/489 [==============================] - 2s 3ms/step - loss: 6501238272.0000 - output_1_loss: 6501165056.0000 - output_2_loss: 6501846016.0000 - output_1_root_mean_squared_error: 80629.8047 - output_2_root_mean_squared_error: 80634.0234 - val_loss: 6416314880.0000 - val_output_1_loss: 6415961088.0000 - val_output_2_loss: 6419501056.0000 - val_output_1_root_mean_squared_error: 80099.6953 - val_output_2_root_mean_squared_error: 80121.7891
Epoch 3/10
489/489 [==============================] - 2s 3ms/step - loss: 6401469952.0000 - 

In [9]:
# Example of logging scalar value
summary_writer = tf.summary.create_file_writer('./my_logs')
with summary_writer.as_default():
    tf.summary.scalar('loss', loss, step=epoch)

NameError: name 'loss' is not defined